In [1]:
from typing import Optional

import torch
from datasets import load_dataset

import transformers
from transformers import AutoTokenizer, HfArgumentParser, AutoModelForSeq2SeqLM, AutoModelForCausalLM

from peft import AutoPeftModelForCausalLM


In [2]:
SUBREDDITS = [
    "askculinary",
    "askhr",
    "askdocs",
    "askanthropology",
    "asksciencefiction",
    "askacademia",
    "askengineers",
    "legaladvice",
    "explainlikeimfive",
    "askbaking",
    "askphysics",
    "askscience",
    "askphilosophy",
    "askvet",
    "changemyview",
    "askcarguys",
    "askhistorians",
    "asksocialscience",
]

SUBREDDITS_NORMVIO = ["askphilosophy", "changemyview", "explainlikeimfive", "legaladvice"]

ANTHROPIC = [
    "anthropic_helpful",
    "anthropic_harmful",
][:1]

SUBREDDIT2DESCRIPTION = {
    "askculinary" : "/r/AskCulinary provides expert guidance for your specific cooking problems to help people of all skill levels become better cooks, to increase understanding of cooking, and to share valuable culinary knowledge.",
    "askhr" : "A place for employees to ask questions about compensation, benefits, harassment, discrimination, legal, and ethical issues in the workplace.",
    "askdocs" : "Having a medical issue? Ask a doctor or medical professional on Reddit! All flaired medical professionals on this subreddit are verified by the mods.",
    "askanthropology" : "Have you ever wanted to know why humans have been so successful as a species? How societies function without governments, laws, or money? What life was like ten thousand years ago? This is the place to ask!",
    "asksciencefiction" : "**It's like Ask Science, but all questions and answers are written with answers gleaned from the universe itself.** Use in-universe knowledge, rules, and common sense to answer the questions. Or as **fanlore.org** calls it [Watsonian, not a Doylist point of view](http://fanlore.org/wiki/Watsonian_vs._Doylist)",
    "askacademia" : "This subreddit is for discussing academic life, and for asking questions directed towards people involved in academia, (both science and humanities).",
    "askengineers" : "Engineers apply the knowledge of math & science to design and manufacture maintainable systems used to solve specific problems. AskEngineers is a forum for questions about the technologies, standards, and processes used to design & build these systems, as well as for questions about the engineering profession and its many disciplines.",
    "legaladvice" : "A place to ask simple legal questions, and to have legal concepts explained.",
    "explainlikeimfive" : "Explain Like I'm Five is the best forum and archive on the internet for layperson-friendly explanations. Don't Panic!",
    "askbaking" : "Welcome to /r/AskBaking! This subreddit is devoted to the discussion of baking, the questions that arise during the process, and requests for critiques or comments on your work!",
    "askphysics" : "A subreddit to draw simple physics questions away from /r/physics. Ask away.",
    "askscience" : "Ask a science question, get a science answer.",
    "askphilosophy" : "/r/askphilosophy aims to provide serious, well-researched answers to philosophical questions.",
    "askvet" : "A place where you can ask veterinary medicine related questions and get advice from veterinary professionals.",
    "changemyview" : "A place to post an opinion you accept may be flawed, in an effort to understand other perspectives on the issue. Enter with a mindset for conversation, not debate.",
    "askcarguys" : "This is a subreddit for automotive related questions.",
    "askhistorians" : "The Portal for Public History. Please read the rules before participating, as we remove all comments which break the rules. Answers must be in-depth and comprehensive, or they will be removed.",
    "asksocialscience" : "The goal of AskSocialScience is to provide great answers to social science questions, based on solid theory, practice, and research.",
}

In [3]:
########## load finetuned models
# model_name = "../../../../llama/hf_model-7B/"
#model_name = "meta-llama/Llama-2-7b-chat-hf"
device=0
if not torch.cuda.is_available():
    device="cpu"  

model_names = {"plain_all": {}, "subredditname_all": {}, "subredditname_askphysics": {}, "subredditname_explainlikeimfive": {}, "contextualized_all": {}}

# plain_all
model_names["plain_all"]["sft"] = "/projects/tir6/general/sachink/personalized-LM/2023/models/0923-newreddit/sft/llama-7B_plain_all/final_checkpoint"
model_names["plain_all"]["dpo"] = "/pprojects/tir6/general/sachink/personalized-LM/2023/models/0923/dpo/llama-7B_plain_all/"
model_names["plain_all"]["ppo"] = "/projects/tir5/users/sachink/personalized-LM/0923/rlhf/llama-se-rl-finetune-128-8-8-1.4e-5_adam_plain_allstep_400"
# subredditname_all
model_names["subredditname_all"]["sft"] = "/projects/tir6/general/sachink/personalized-LM/2023/models/0923-newreddit/sft/llama-7B_subredditname_all/final_checkpoint"
model_names["subredditname_all"]["dpo"] = "/pprojects/tir6/general/sachink/personalized-LM/2023/models/0923/dpo/llama-7B_subredditname_all/"
model_names["subredditname_all"]["ppo"] = "/projects/tir5/users/sachink/personalized-LM/0923/rlhf/llama-se-rl-finetune-128-8-8-1.4e-5_adam_subredditname_allstep_400"
# subredditname_askphysics
model_names["subredditname_askphysics"]["sft"] = "/projects/tir6/general/sachink/personalized-LM/2023/models/0923-newreddit/sft/llama-7B_subredditname_askphysics/final_checkpoint"
model_names["subredditname_askphysics"]["dpo"] = "/pprojects/tir6/general/sachink/personalized-LM/2023/models/0923/dpo/llama-7B_subredditname_askphysics/"
model_names["subredditname_askphysics"]["ppo"] = "/projects/tir5/users/sachink/personalized-LM/0923/rlhf/llama-se-rl-finetune-128-8-8-1.4e-5_adam_subredditname_askphysicsstep_400"
# subredditname_explainlikeimfive 
model_names["subredditname_explainlikeimfive"]["sft"] = "/projects/tir6/general/sachink/personalized-LM/2023/models/0923-newreddit/sft/llama-7B_subredditname_explainlikeimfive/final_checkpoint"
model_names["subredditname_explainlikeimfive"]["dpo"] = "/pprojects/tir6/general/sachink/personalized-LM/2023/models/0923/dpo/llama-7B_subredditname_explainlikeimfive/"
model_names["subredditname_explainlikeimfive"]["ppo"] = "/projects/tir5/users/sachink/personalized-LM/0923/rlhf/llama-se-rl-finetune-128-8-8-1.4e-5_adam_subredditname_explainlikeimfivestep_400"
# contextualized_all
model_names["contextualized_all"]["sft"] = "/projects/tir6/general/sachink/personalized-LM/2023/models/0923-newreddit/sft/llama-7B_contextualized_all/final_checkpoint"
model_names["contextualized_all"]["dpo"] = "/pprojects/tir6/general/sachink/personalized-LM/2023/models/0923/dpo/llama-7B_contextualized_all/"
model_names["contextualized_all"]["ppo"] = "/projects/tir5/users/sachink/personalized-LM/0923/rlhf/llama-se-rl-finetune-128-8-8-1.4e-5_adam_contextualized_allstep_400"

instrtype = "subredditname"
subset = "all"
algorithm = "sft"

model_name = model_names[f'{instrtype}_{subset}'][f"{algorithm}"]
model = AutoPeftModelForCausalLM.from_pretrained(model_name, load_in_8bit=True)    
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")

generation_kwargs = {"top_k": 0.0, "top_p": 0.95, "do_sample": True, "eos_token_id": -1}


model.to(device)

model.device

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

device(type='cuda', index=0)

In [1]:
from peft import PeftConfig, PeftModel

# plain_all
base_model_name = "/projects/tir6/general/sachink/personalized-LM/2023/models/0923/sft/llama-7B_plain_all/final_checkpoint"
adapter_model_name = "/projects/tir6/general/sachink/personalized-LM/2023/models/0923/dpo/llama-7B_plain_all/final_checkpoint"
# subredditname_all
base_model_name = "/projects/tir6/general/sachink/personalized-LM/2023/models/0923/sft/llama-7B_subredditname_all/final_checkpoint"
adapter_model_name = "/projects/tir6/general/sachink/personalized-LM/2023/models/0923/dpo/llama-7B_subredditname_all/final_checkpoint"
# subredditname_askphysics
# base_model_name = "/projects/tir6/general/sachink/personalized-LM/2023/models/0923/sft/llama-7B_subredditname_askphysics/final_checkpoint"
# adapter_model_name = "/projects/tir6/general/sachink/personalized-LM/2023/models/0923/dpo/llama-7B_subredditname_askphysics/final_checkpoint"
# subredditname_explainlikeimfive 
# base_model_name = "/projects/tir6/general/sachink/personalized-LM/2023/models/0923/sft/llama-7B_subredditname_explainlikeimfive/final_checkpoint"
# adapter_model_name = "/projects/tir6/general/sachink/personalized-LM/2023/models/0923/dpo/llama-7B_subredditname_explainlikeimfive/final_checkpoint"
# contextualized_all
# base_model_name = "/projects/tir6/general/sachink/personalized-LM/2023/models/0923/sft/llama-7B_contextualized_all/final_checkpoint"
# adapter_model_name = "/projects/tir6/general/sachink/personalized-LM/2023/models/0923/dpo/llama-7B_contextualized_all/final_checkpoint"
peft_config = PeftConfig.from_pretrained(adapter_model_name)

model = AutoPeftModelForCausalLM.from_pretrained(
    base_model_name, return_dict=True, torch_dtype=torch.bfloat16
)

# tokenizer = AutoTokenizer.from_pretrained(base_model_name)
device=0
if not torch.cuda.is_available():
    device="cpu"  


# # Load the Lora model
model = PeftModel.from_pretrained(model, adapter_model_name).half().to(device)
model.eval()

# model = model.merge_and_unload()

# model.save_pretrained(f"{script_args.output_name}")
# tokenizer.save_pretrained(f"{script_args.output_name}")
# model.push_to_hub(f"{script_args.output_name}", use_temp_dir=False)



NameError: name 'AutoPeftModelForCausalLM' is not defined

In [7]:
# n = int(input("Output length: " ))
# instrtype = "plain"
# domain = "askphysics"
domain = "legaladvice"
if instrtype == "subredditname":
    instruction = f"Write a response to this reddit post in the following subreddit. SUBREDDIT: {domain}. \n\n POST: "
elif instrtype == "contextualized":
    instruction = f"Write a response to this reddit post in the subreddit with the following description. SUBREDDIT: {SUBREDDIT2DESCRIPTION[domain]}. \n\n POST: "
else:
    instruction = f"Write a response to this reddit post. \n\n POST: "

n = 128

# lines = ""
# no_of_lines = 1
# print(f"Enter new prompt ({no_of_lines} lines): ")
# for i in range(no_of_lines):
#     lines += input()+"\n\n"
lines = ["What is the purpose of life?", "Why is euthanasia illegal?"]
    
# input(batch)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"
query_tensors = tokenizer([instruction+line+" \n\n COMMENT: " for line in lines], padding=True, return_tensors="pt").to(device)
print(query_tensors)




{'input_ids': tensor([[    2,     1, 14350,   263,  2933,   304,   445,   337,  1289,   277,
          1400,   297,   278,  1494,  1014,  1127, 27423, 29889, 27092,  1525,
          7858,  1806, 29901, 11706,   328,  1087, 29889, 29871,    13,    13,
         11971, 29901,  1724,   338,   278,  6437,   310,  2834, 29973, 29871,
            13,    13,  4810,  7428,  3919, 29901, 29871],
        [    1, 14350,   263,  2933,   304,   445,   337,  1289,   277,  1400,
           297,   278,  1494,  1014,  1127, 27423, 29889, 27092,  1525,  7858,
          1806, 29901, 11706,   328,  1087, 29889, 29871,    13,    13, 11971,
         29901,  3750,   338,   321,  2806, 16397,   423, 27302, 29973, 29871,
            13,    13,  4810,  7428,  3919, 29901, 29871]], device='cuda:0'), 'attention_mask': tensor([[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [8]:


# query_tensors = tokenizer.encode(lines, return_tensors="pt").to(device)
print("Output: ", end="")
# Get response from t5
response_tensor = model.generate(**query_tensors, max_new_tokens=n, **generation_kwargs)
response = tokenizer.decode(response_tensor[0], skip_special_tokens=True)

print(response)

Output: 

Write a response to this reddit post in the following subreddit. SUBREDDIT: legaladvice. 

 POST: What is the purpose of life? 

 COMMENT:  When we run out of resources, we die.  When those around us run out of resources we die.  And that's how it works for every species.  Being lazy and thoughtful enough to convince ourselves that we are a special type of life form that does not have to die off is... well, I believe it's what we call "Arrogance".  If we could be honest with ourselves and look at our species in the same way we look at other species in the ecosystem then we might actually make some progress on the issues we're facing.   Example: Global Warming.


In [9]:
response = tokenizer.decode(response_tensor[1], skip_special_tokens=False)

print(response)

<s> Write a response to this reddit post in the following subreddit. SUBREDDIT: legaladvice. 

 POST: Why is euthanasia illegal? 

 COMMENT:  Volunteer as a rescue dog handler. It’s a weird word salad way of saying you're a dog adoption social worker. Many people who can’t have dogs for whatever reason seek out rescue dogs as their companions. Some of them are great dogs. The volunteer goes out once a month to the shelters, kennels, stray pickups and all sorts of different places to see which dogs have just been saved from the street. Clean, vaccinated and ready to get the new loving home that they didn’t end up with. A lot of them are happy
